In [1]:
!pip -q install gseapy

     |████████████████████████████████| 525 kB 5.1 MB/s 
     |████████████████████████████████| 199 kB 67.3 MB/s 
     |████████████████████████████████| 44 kB 2.3 MB/s 
     |████████████████████████████████| 47 kB 4.1 MB/s 
     |████████████████████████████████| 255 kB 65.9 MB/s 
     |████████████████████████████████| 5.8 MB 37.3 MB/s 
     |████████████████████████████████| 251 kB 48.0 MB/s 
     |████████████████████████████████| 127 kB 66.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [1]:
import pandas as pd
import gseapy as gp
from scipy.optimize import nnls


def rawEnrichmentAnalysis(expr):
  # Reduce the expression dataset to contain only the required genes
  genes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/genes.csv', index_col=0)
  expr = expr.loc[expr.index.intersection(list(genes.x))]
  if (len(expr.index) < 5000):
    raise("ERROR: not enough genes")

  # Transform the expression to rank
  for col in expr:
    expr[col] = expr[col].rank()

  # Run ssGSEA analysis for the ranked gene expression dataset
  # txt, gct file input
  gene_sets = {}

  with open('/content/drive/MyDrive/Colab Notebooks/signatures.txt') as f:
    for line in f:
      key, val = line.split('\t', 1)
      val = val.split()
      gene_sets[key] = val

  print(f'Number of samples: {len(expr.columns)}, number of gene sets: {len(gene_sets)}')

  ssg = gp.ssgsea(data=expr,
                gene_sets=gene_sets, #gene_sets={'A':['gene1', 'gene2',...], 'B':['gene2', 'gene4',...],  ...}
                #outdir='test/ssgsea_report',
                sample_norm_method='custom', # choose 'custom' for your own rank list
                permutation_num=0, # skip permutation procedure, because you don't need it
                no_plot=True, # skip plotting, because you don't need these figures
                processes=20,
                seed=9,
                scale=False)

  scores = pd.DataFrame(ssg.resultsOnSamples)

  # Rescale on gene sets.
  scores = scores.subtract(scores.min(axis='columns'), axis='rows')

  # Combine signatures for same cell types
  scores['cell_type'] = scores.index.str.split('%', 1)
  scores['cell_type'] = scores.cell_type.apply(lambda x: x[0])
  scores = scores.groupby('cell_type').mean()
  return scores

In [45]:
expr = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sdy_expr420.csv', index_col=0)
#expr = expr.iloc[:,:10]
scores = rawEnrichmentAnalysis(expr)

fv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/spill_fv.csv', index_col=0)
fv = fv.reindex(scores.index)

#Rescale on cell types.
scores = scores.subtract(scores.min(axis='columns'), axis='rows')
scores = scores/5000

#Use fv formula tscores <- (tscores^fit.vals[A,2])/(fit.vals[A,3]*2).
scores = scores.pow(fv.V2, axis='rows')
scores = scores.divide(fv.V3*2, axis='rows')

#scores = spillOver(transformed.scores,xCell.data$spill.array$K)
K = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/spill_K.csv', index_col=0)
K = K.reindex(scores.index)
K = K[K.index]
alpha = 0.5
K = K * alpha

def spillOver(sample):
  #Apply correction on samples: scores <- apply(transformedScores[rows, ], 2, function(x) pracma::lsqlincon(K[rows,rows], x, lb = 0)).
  x, rnorm = nnls(K, sample)
  return x

scores = scores.apply(lambda x: spillOver(x), axis='rows')

Number of samples: 104, number of gene sets: 489
